In [1]:
import os
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project")
print(os.getcwd())

/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project


In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as ps
import glob
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [3]:
states = pd.read_csv("States.csv")
geofips = pd.read_csv("geofips.csv")

In [4]:
def cap(string):
    if isinstance(string, str):
        return string.upper().replace(" COUNTY", "").capitalize().strip()
    else:
        return ""

In [5]:
#--------------------READING AND PRE-PROCESSING PER CAPITA INCOME DATA--------------------
#-----------------------------------------------------------------------------------------

#Read in per capita income
xls = pd.ExcelFile("data/download.xls")
income_raw = xls.parse("Sheet0", skiprows=5, index_col=None, na_values=[np.nan], skip_footer=12)

#display(data_test[2200:2400])
#display(data_test[:3138].tail()) #anything beyond row 3137 are footnotes

income_raw.head()

,GeoFips,GeoName,2010,2011,2012,2013,2014,2015,2016
0,01001,"Autauga, AL",33402,34302,35048,35503,36696,38575,39721
1,01003,"Baldwin, AL",36089,37474,37637,37723,38939,40640,41286
2,01005,"Barbour, AL",27916,28099,28065,30120,30166,31635,31788
3,01007,"Bibb, AL",25003,25935,26995,27388,28123,28919,29264
4,01009,"Blount, AL",27599,28135,29242,29840,30949,31560,31470


In [6]:
data_test = income_raw.copy()

#Extract state and county from GeoName
data_test = data_test[:3138]

data_test["Abbreviation"] = data_test["GeoName"].apply(lambda x: x.split(",")[1].replace(" ", ""))
data_test["County"] = data_test["GeoName"].apply(lambda x: x.split(",")[0])
data_test["Abbreviation"] = data_test["Abbreviation"].str.replace("*","")
#display(data_test[60:70])

#Virginia is a problem child:
data_test["Abbreviation"] = data_test["Abbreviation"].replace(["Staunton+Waynesboro","ColonialHeights+Petersburg","FairfaxCity+FallsChurch","Manassas+ManassasPark","BuenaVista+Lexington"],"VA")

#Get full state name from abbreviation
data_test = pd.merge(data_test, states, how="left", on=["Abbreviation"]).drop(["GeoName", "Abbreviation"], axis=1)
data_test.head()

#Replace NAs
data_test = data_test.replace('(NA)',np.nan)

#Replace Sout with South Dakota
data_test['State'] = data_test['State'].str.replace('Sout ','South Dakota')

#Split up combined counties
data_test["County"] = data_test["County"].apply(lambda x: x.split(" + ")[0])

#Get rid of notes
data_test['County'] = data_test['County'].str.replace('\(.+\)','')

#Update counties
data_test['GeoFips'] = pd.to_numeric(data_test['GeoFips'], downcast='integer')
data_test = pd.merge(data_test, geofips, on='GeoFips', how='left')
data_test['County_y'] = data_test['County_y'].fillna(data_test['County_x'].str.lower())

#Update county and state columns
data_test.drop(['County_x', 'State_y'], axis=1, inplace=True)
data_test.rename({'County_y':'County', 'State_x':'State'}, axis=1, inplace=True)
data_test['State'] = data_test['State'].str.lower()
display(data_test[66:70])

,GeoFips,2010,2011,2012,2013,2014,2015,2016,State,County
66,1133,27908.0,28137.0,29180.0,29725.0,31449.0,32573.0,33215.0,alabama,winston
67,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alaska,aleutian islands census area
68,2013,35949.0,37727.0,37213.0,39624.0,41051.0,50926.0,60284.0,alaska,aleutians east borough
69,2016,41086.0,45850.0,50427.0,50195.0,52392.0,52515.0,56105.0,alaska,aleutians west census area


In [7]:
#data_test.to_csv("./cleaned_data/income_test.csv", index=None, header=True)

In [8]:
#Transposing against year
for i in range(7):
    income = data_test.iloc[:, [9, 8] + [i + 1]]
    income["Year"] = 2010 + i
    income.columns = ["State", "County", "Income", "Year"]
    
    if i == 0:
        data3 = income
    else:
        data3 = pd.concat([data3, income])  

#Removing trailing blanks from county and state variables and make lower case
data3["State"] = data3["State"].apply(cap).str.lower()
data3["County"] = data3["County"].apply(cap).str.lower()

In [9]:
income = data3
income.head()

income.rename({'State':'County', 'County':'State'}, axis=1, inplace=True)

In [10]:
income.head()

,County,State,Income,Year
0,autauga,alabama,33402.0,2010
1,baldwin,alabama,36089.0,2010
2,barbour,alabama,27916.0,2010
3,bibb,alabama,25003.0,2010
4,blount,alabama,27599.0,2010


In [11]:
#Checking Number of States
query = """SELECT DISTINCT State FROM income ORDER BY State ASC"""
ps.sqldf(query=query)

,State
0,alabama
1,alaska
2,arizona
3,arkansas
4,california
5,colorado
6,connecticut
7,delaware
8,florida
9,georgia


In [12]:
query = """SELECT * FROM income WHERE Year = 2016"""
income_2016 = ps.sqldf(query=query)

In [13]:
data3.to_csv("./cleaned_data/income_all-years.csv", index=None, header=True)
data3.head()

income_2016.to_csv("./cleaned_data/income_2016.csv", index=None, header=True)